In [94]:
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d as p3
import numpy as np
import random
import time


from functools import partial
from ipywidgets import interact, RadioButtons, IntSlider, FloatSlider, Dropdown, BoundedFloatText
from numpy.linalg import norm

random.seed(42) # начальное состояние генератора случайных чисел, чтобы можно было воспроизводить результаты.

#Активационная функция сигмоидального нейрона
def sigmoid(x):
    return 1/(1+np.exp(-x))

#Производная активационной функции сигмоидального нейрона
def sigmoid_prime(x):
    return sigmoid(x) * (1 - sigmoid(x))


#Считает целевую функциюю
def J_quadratic(neuron, X, y):
    """
    Оценивает значение квадратичной целевой функции.
    Всё как в лекции, никаких хитростей.

    neuron - нейрон, у которого есть метод vectorized_forward_pass, предсказывающий значения на выборке X
    X - матрица входных активаций (n, m)
    y - вектор правильных ответов (n, 1)

    Возвращает значение J (число)
    """

    assert y.shape[1] == 1, 'Incorrect y shape'

    return 0.5 * np.mean((neuron.vectorized_forward_pass(X) - y) ** 2)
def J_by_weights(weights, X, y, bias):
    """
    Посчитать значение целевой функции для нейрона с заданными весами.
    Только для визуализации
    """
    new_w = np.hstack((bias, weights)).reshape((3,1))
    return J_quadratic(Neuron(new_w), X, y)



#Провера градиента(численная производная целевой функции)
def compute_grad_numerically_2(neuron, X, y, J=J_quadratic, eps=10e-2):
    
    w_0 = neuron.w
    num_grad = np.zeros(neuron.w.shape)

    for i in range(len(neuron.w)):
        
        for k in range(-1,2,2):
            
            oldW = neuron.w[i,0]
            neuron.w[i] += eps*k
            num_grad[i] += J(neuron, X, y)*k
            neuron.w[i,0] = oldW
        
    assert np.allclose(neuron.w, w_0), "WE SPOILED THE WEIGHT"
    return num_grad/(2*eps)

   
class Neuron:

    def __init__(self, weights, activation_function=sigmoid, activation_function_derivative = sigmoid_prime):
        
        assert weights.shape[1] == 1, "Incorrect weight shape"
        
        self.w = weights
        self.activation_function = activation_function
        self.activation_function_derivative = activation_function_derivative
    
    #Сумматорный метод
    def summatory(self, input_matrix):
        return input_matrix.dot(self.w)
    
    
    #Активационный метод
    def activation(self, summatory_activation):
        return self.activation_function(summatory_activation)
    
    
    #Векторизованный метод рассчета предсказанных значений(гораздо лучший способ)
    def vectorized_forward_pass(self, input_matrix):
        return self.activation(self.summatory(input_matrix))

    
    #Обучение весов с помощью одного батча(несколько случайных примеров из всех имеющихся примеров)
    def update_mini_batch(self, X, y, learning_rate, eps):
        
        before = J_quadratic(self, X, y)
        
        #self.w -= learning_rate * compute_grad_analytically(self,X,y)
        self.w -= learning_rate * compute_grad_numerically_2(self,X,y)

        
        return np.abs(before-J_quadratic(self, X, y)) < eps
    
    
    #Обучение весов с помощью батчей
    def SGD(self, X, y, batch_size, learning_rate =0.1, eps=1e-6, max_steps = 200):
        cur_step = 0
        #for _ in range(max_steps):
        while cur_step < max_steps:
            cur_step += 1
            batch = np.random.choice(len(X), batch_size, replace = False)
            if self.update_mini_batch(X[batch], y[batch], learning_rate, eps) == 1: 
                return 1
        return 0




#Загрузка данных из файла
data = np.loadtxt("data.csv", delimiter=",")

# Подготовим данные

X = data[:, :-1]
y = data[:, -1]

X = np.hstack((np.ones((len(y), 1)), X))
y = y.reshape((len(y), 1)) # Обратите внимание на эту очень противную и важную строчку


# Создадим нейрон

w = np.random.random((X.shape[1], 1))
neuron = Neuron(w, activation_function=sigmoid, activation_function_derivative=sigmoid_prime)

    
%matplotlib inline



#point = (neuron.w[1,0], neuron.w[2,0])
point = (0,20)
redPoint = (point[0], point[1], J_by_weights(point, X, y, 0.0))
max_b = 40
min_b = -40
max_w1 = 40
min_w1 = -40
max_w2 = 40
min_w2 = -40

g_bias = 0 # график номер 2 будет при первой генерации по умолчанию иметь то значение b, которое выставлено в первом
X_corrupted = X.copy()
y_corrupted = y.copy()



#Начало реализации истории обучения 
@interact(b=BoundedFloatText(value=str(g_bias), min=min_b, max=max_b, description="Enter $b$:"),
          w1=BoundedFloatText(value=redPoint[0], min=min_w1, max=max_w1, description="Enter $w_1$:"),
          w2=BoundedFloatText(value=redPoint[1], min=min_w2, max=max_w2, description="Enter $w_2$:"),
          learning_rate=Dropdown(options=["0.01", "0.05", "0.1", "0.5", "1", "5", "10"], 
                                value="10", description="Learning rate: ")
         )
def learning_curve_for_starting_point(b, w1, w2, learning_rate=0.1):
    w = np.array([b, w1, w2]).reshape(X_corrupted.shape[1], 1)
    learning_rate=float(learning_rate)
    neuron = Neuron(w, activation_function=sigmoid, activation_function_derivative=sigmoid_prime)

    story = [J_quadratic(neuron, X_corrupted, y_corrupted)]
    for _ in range(2000):
        neuron.SGD(X_corrupted, y_corrupted, 2, learning_rate=learning_rate, max_steps=2)
        story.append(J_quadratic(neuron, X_corrupted, y_corrupted))
    plt.plot(story)
    
    plt.title("Learning curve.\n Final $b={0:.3f}$, $w_1={1:.3f}, w_2={2:.3f}$".format(*neuron.w.ravel()))
    plt.ylabel("$J(w_1, w_2)$")
    plt.xlabel("Weight and bias update number")
    plt.show()
    

#Начало визуализации целевой функции
@interact(fixed_bias=FloatSlider(min=min_b, max=max_b, continuous_update=False), 
          mixing=FloatSlider(min=0, max=1, continuous_update=False, value=0),
          shifting=FloatSlider(min=0, max=1, continuous_update=False, value=0),
          w1n=FloatSlider(min=-40, max=40, continuous_update=False, value=neuron.w[1,0]),
          w2n=FloatSlider(min=-40, max=40, continuous_update=False, value=neuron.w[2,0]),
         )
def visualize_cost_function(fixed_bias, mixing, shifting, w1n, w2n):
    """
    Визуализируем поверхность целевой функции на (опционально) подпорченных данных и сами данные.
    Портим данные мы следующим образом: сдвигаем категории навстречу друг другу, на величину, равную shifting 
    Кроме того, меняем классы некоторых случайно выбранных примеров на противоположнее.
    Доля таких примеров задаётся переменной mixing
    
    Нам нужно зафиксировать bias на определённом значении, чтобы мы могли что-нибудь визуализировать.
    Можно посмотреть, как bias влияет на форму целевой функции
    """

    xlim = (min_w1, max_w1)
    ylim = (min_w2, max_w2)
    xx = np.linspace(*xlim, num=101)
    yy = np.linspace(*ylim, num=101)
    xx, yy = np.meshgrid(xx, yy)
    points = np.stack([xx, yy], axis=2)
    
    # не будем портить исходные данные, будем портить их копию
    corrupted = data.copy()
    
    # инвертируем ответы для случайно выбранного поднабора данных
    mixed_subset = np.random.choice(range(len(corrupted)), int(mixing * len(corrupted)), replace=False)
    corrupted[mixed_subset, -1] = np.logical_not(corrupted[mixed_subset, -1])
    
    # сдвинем все груши (внизу справа) на shifting наверх и влево
    pears = corrupted[:, 2] == 1
    apples = np.logical_not(pears)
    corrupted[pears, 0] -= shifting
    corrupted[pears, 1] += shifting
    
    # вытащим наружу испорченные данные
    global X_corrupted, y_corrupted
    X_corrupted = np.hstack((np.ones((len(corrupted),1)), corrupted[:, :-1]))
    y_corrupted = corrupted[:, -1].reshape((len(corrupted), 1))
    
    
    #point = (neuron.w[1,0], neuron.w[2,0])
    bluePoint = (point[0], point[1], J_by_weights(point, X_corrupted, y_corrupted, fixed_bias))
    redPoint = (w1n, w2n, J_by_weights((w1n,w2n), X_corrupted, y_corrupted, fixed_bias))
    # посчитаем значения целевой функции на наших новых данных
    calculate_weights = partial(J_by_weights, X=X_corrupted, y=y_corrupted, bias=fixed_bias)
    J_values = np.apply_along_axis(calculate_weights, -1, points)
    
    fig = plt.figure(figsize=(16,5))
    # сначала 3D-график целевой функции
    ax_1 = fig.add_subplot(1, 2, 1, projection='3d')
    surf = ax_1.plot_surface(xx, yy, J_values, alpha=0.3)
    ax_1.plot_wireframe(xx, yy, J_values, rcount=11, ccount=11, color = "black")
    ax_1.scatter(redPoint[0], redPoint[1], redPoint[2],  s = 200, color = "red", edgecolors = "b")
    ax_1.scatter(bluePoint[0], bluePoint[1], bluePoint[2],  s = 200, color = "blue", edgecolors = "b")
    ax_1.set_xlabel("$w_1$")
    ax_1.set_ylabel("$w_2$")
    ax_1.set_zlabel("$J(w_1, w_2)$")
    ax_1.set_title("$J(w_1, w_2)$ for fixed bias = ${}$".format(fixed_bias))
    # потом плоский поточечный график повреждённых данных
    ax_2 = fig.add_subplot(1, 2, 2)
    plt.scatter(corrupted[apples][:, 0], corrupted[apples][:, 1], color = "red", alpha=0.7)
    plt.scatter(corrupted[pears][:, 0], corrupted[pears][:, 1], color = "green", alpha=0.7)
    ax_2.set_xlabel("yellowness")
    ax_2.set_ylabel("symmetry")

    plt.show()



interactive(children=(BoundedFloatText(value=0.0, description='Enter $b$:', max=40.0, min=-40.0), BoundedFloat…

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='fixed_bias', max=40.0, min=…